In [1]:
import datetime
import pytz
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import matplotlib.ticker as mpticker
import pandas as pd
import numpy as np

from pathlib import Path
from mpl_finance import candlestick_ohlc

C:\Anaconda3\envs\dev\lib\site-packages\mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


#### Download Cryptocurrency trading data

In [2]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    Path('../Resources/BTCUSD_1h.csv'),
    index_col="Date")
df.tail()

,Open,Close,High,Low,Volume
Date,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020


#### Using hierarchical clustering -- Agglomerative Clustering approach on stocks rolling data

In [26]:
from sklearn.cluster import AgglomerativeClustering

# Create rolling data of 3 hourss each for both max and min value
max = df['High'].rolling(3).max().rename('waves')
min = df['High'].rolling(3).min().rename('waves')
min.tail(20)

Date
2021-11-16 22:00:00    60203.000000
2021-11-16 23:00:00    60657.000000
2021-11-17 00:00:00    60409.000000
2021-11-17 01:00:00    60258.000000
2021-11-17 02:00:00    59733.545599
2021-11-17 03:00:00    59733.545599
2021-11-17 04:00:00    59503.000000
2021-11-17 05:00:00    59503.000000
2021-11-17 06:00:00    59503.000000
2021-11-17 07:00:00    59783.000000
2021-11-17 08:00:00    59783.000000
2021-11-17 09:00:00    59619.000000
2021-11-17 10:00:00    59619.000000
2021-11-17 11:00:00    59619.000000
2021-11-17 12:00:00    60621.000000
2021-11-17 13:00:00    60639.000000
2021-11-17 14:00:00    60388.000000
2021-11-17 15:00:00    59859.000000
2021-11-17 16:00:00    59859.000000
2021-11-17 17:00:00    59859.000000
Name: waves, dtype: float64

In [27]:
# Prepare data in two column format
maxdf = pd.concat([max,pd.Series(np.zeros(len(max))+1)],axis = 1)
mindf = pd.concat([min,pd.Series(np.zeros(len(min))+-1)],axis = 1)

# Drop duplications
maxdf.drop_duplicates('waves',inplace = True)
mindf.drop_duplicates('waves',inplace = True)

mindf.tail()

,waves,0
2021-11-17 07:00:00,59783.0,NaN
2021-11-17 09:00:00,59619.0,NaN
2021-11-17 13:00:00,60639.0,NaN
2021-11-17 14:00:00,60388.0,NaN
2021-11-17 15:00:00,59859.0,NaN


In [38]:
# Use n_cluster = 3
F = maxdf.append(mindf).sort_index()
F = F[ F[0] != F[0].shift()].dropna()

# Find Support/Resistance with clustering

# Create [x,y] array where y is always 1
X = np.concatenate((F.waves.values.reshape(-1,1),
                    (np.zeros(len(F))+1).reshape(-1,1)), axis = 1)

X

array([], shape=(0, 2), dtype=float64)

In [39]:
# Pick n_clusters (the sqrt of df + 2)
n = round(len(F)**(1/2)) + 2 
cluster = AgglomerativeClustering(n_clusters=n,
            affinity='euclidean', linkage='ward')
cluster.fit_predict(X)
F['clusters'] = cluster.labels_

F2 = F.loc[F.grouby('clusters')['waves'].idxmax()]

# Plotit
fig, axis = plt.subplots()
for row in F2.itertuples():

    axis.axhline( y = row.waves,
            color = 'green', ls = 'dashed')

axis.plot( F.index.values, F.waves.values)
plt.show()

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 2 is required by AgglomerativeClustering.